In [3]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, FileLink
from io import StringIO
import csv

# Text area widget for raw TSV input
order_input = widgets.Textarea(
    value='Reference #\tPick ID\tPick Date\tSales Order No.\tCustomer PO\tShip To\tStreet\tCity\tZip Code\tCountry/Region\tItem No.\tEach Qty\tstate\tCLIENT\tWHSE\n'
          '290638-71387\t71387\t08/11/2025\t290638\tPO#93099\tFortin\'s Supply Ltd\t45750 Airport Road\tChilliwack\tV2P 1A2\tCA\tCH6590\t1\tBC\tKL04\tFRASER\n'
          'nan\tnan\tnan\tnan\tnan\tnan\tnan\tnan\tnan\tnan\tnan\tnan\tnan\tnan\tnan\n'
          '290638-71387\t71387\t08/11/2025\t290638\tPO#93099\tFortin\'s Supply Ltd\t45750 Airport Road\tChilliwack\tV2P 1A2\tCA\tCH8950\t3\tBC\tKL04\tFRASER',
    description='Paste your tabular data (TSV):',
    layout=widgets.Layout(width='100%', height='300px')
)

generate_button = widgets.Button(description='Generate & Download CSV (no headers)', button_style='success')
output = widgets.Output()

csv_filename = 's_orders_output.csv'

def trim_text(val, max_len):
    if pd.isnull(val):
        return ''
    return str(val).strip()[:max_len]

def is_present(val):
    s = str(val).strip()
    return s != '' and s.lower() != 'nan'

def on_generate_clicked(b):
    with output:
        output.clear_output()
        raw_text = order_input.value.strip()
        if not raw_text:
            print("⚠️ Please paste your tabular data (with headers).")
            return
        try:
            df = pd.read_csv(StringIO(raw_text), delimiter='\t', dtype=str)
        except Exception as e:
            print("❌ Error reading data:", e)
            return

        needed_cols = ['Sales Order No.', 'Pick Date', 'Item No.', 'Each Qty', 'WHSE', 'Ship To', 'Street', 'City', 'Zip Code', 'Country/Region', 'state', 'Customer PO', 'CLIENT']
        missing = [c for c in needed_cols if c not in df.columns]
        if missing:
            print(f"❌ Missing columns in input: {missing}")
            return

        for col in needed_cols:
            df[col] = df[col].fillna('').astype(str).str.strip()

        df['Pick Date'] = pd.to_datetime(df['Pick Date'], errors='coerce').dt.strftime('%m/%d/%Y').fillna('')

        cols_A_to_Z = [chr(i) for i in range(ord('A'), ord('Z')+1)]
        cols_AA_to_AJ = ['A'+chr(i) for i in range(ord('A'), ord('J')+1)]
        all_cols = cols_A_to_Z + cols_AA_to_AJ

        valid_rows = []
        invalid_found = False
        for idx, row in df.iterrows():
            has_ref = is_present(row['Sales Order No.'])
            has_item = is_present(row['Item No.'])
            if has_ref and has_item:
                out_row = {col: '' for col in all_cols}
                out_row['A'] = 'BC'
                out_row['B'] = trim_text(row['CLIENT'], 45)
                out_row['C'] = trim_text(row['Sales Order No.'], 45)
                out_row['D'] = trim_text(row['Customer PO'], 45)
                out_row['F'] = row['Pick Date']
                out_row['G'] = row['Pick Date']
                out_row['I'] = trim_text(row['Ship To'], 45)
                out_row['K'] = trim_text(row['Street'], 30)
                out_row['M'] = trim_text(row['City'], 45)
                out_row['N'] = trim_text(row['state'], 10)
                out_row['O'] = trim_text(row['Zip Code'], 10)
                out_row['P'] = trim_text(row['Country/Region'], 10)
                out_row['X'] = trim_text(row['Item No.'], 45)
                qty_val = pd.to_numeric(row['Each Qty'], errors='coerce')
                out_row['Y'] = int(qty_val) if not pd.isna(qty_val) else 0
                out_row['AJ'] = trim_text(row['WHSE'], 45)
                valid_rows.append(out_row)
            else:
                invalid_found = True

        if not valid_rows:
            print("⚠️ No valid data rows found.")
            return

        out_df = pd.DataFrame(valid_rows, columns=all_cols)

        if invalid_found:
            first_row = out_df.iloc[0].copy()
            first_row['C'] = 'DELETE_ME_ROW'
            out_df = pd.concat([out_df, pd.DataFrame([first_row])], ignore_index=True)

        # Save CSV with Windows-1252 encoding and proper line endings
        csv_content = out_df.to_csv(index=False, header=False, sep=',', encoding='cp1252',
                                   quoting=csv.QUOTE_MINIMAL)
        csv_content = csv_content.replace('\n', '\r\n')

        with open(csv_filename, 'w', encoding='cp1252', newline='') as f:
            f.write(csv_content)

        print(f"✅ CSV generated: {csv_filename}")
        display(FileLink(csv_filename))

generate_button.on_click(on_generate_clicked)

display(order_input, generate_button, output)


Textarea(value="Reference #\tPick ID\tPick Date\tSales Order No.\tCustomer PO\tShip To\tStreet\tCity\tZip Code…

Button(button_style='success', description='Generate & Download CSV (no headers)', style=ButtonStyle())

Output()